In [ ]:
import pandas as pd
import json
import datetime
import time
import os
from corr_plot import corrplot
import numpy as np
from lgbm_consumption_module import data_preprocessing_interventions, lgbm_regression_efecto_acumulado_con_linea_base_del_experimento
import matplotlib
from auxiliary import week_of_month, graph_check_cumulative_bsts, graph_check_gbm_dist, graph_check_gbm_timeseries, prepare_data_synthetic_bsts, prepare_data_control_bsts
from sklearn.model_selection import GroupShuffleSplit,TimeSeriesSplit
import seaborn as sns
from sklearn.metrics import mean_squared_error
import shap
import warnings
warnings.filterwarnings('ignore')
from causalimpact import CausalImpact
from matplotlib import pyplot as plt
from scipy.signal import savgol_filter as sg

In [ ]:
data_mean = data_preprocessing_interventions()

In [ ]:
def add_sg(df, group, lag, diff):
    p = 3
    df['ISO_kWh_smooth_'+str(lag)] = 0.0
    df['SG_kWh_smooth_'+str(lag)] = 0.0
    df['DEG_C_kWh_smooth_'+str(lag)] = 0.0
#     df['ISO_kWh_diff_'+str(lag)] = 0.0
#     df['SG_kWh_diff_'+str(lag)] = 0.0
#     df['DEG_C_kWh_diff_'+str(lag)] = 0.0
    for si in df[group].unique():
        index = df[group] == si
        df.loc[index, 'ISO_kWh_smooth_'+str(lag)] = sg(df[index].ISO_kWh, lag, p)
        df.loc[index, 'SG_kWh_smooth_'+str(lag)] = sg(df[index].SG_kWh, lag, p)
        df.loc[index, 'DEG_C_kWh_smooth_'+str(lag)] = sg(df[index].DEG_C_kWh, lag, p)
    return df
#data_mean = add_sg(data_mean, 'INTERVENTION', 7, diff=1)
data_mean = add_sg(data_mean, 'INTERVENTION', 29, diff=1)
#data_mean = add_sg(data_mean, 'INTERVENTION', 89, diff=1)

In [ ]:
def add_lags(data, lag):
    
    df = data.copy()
    df = df.set_index(['timestamp', 'INTERVENTION'])  # index
    df = df.unstack().shift(lag)  # pull out the groups, shift with lag step=1
    df = df.stack(dropna=False)
    df.reset_index(inplace=True)  # stack the groups back, keep the missing values
    
    data['ISO_kWh_'+str(lag)] = df['ISO_kWh'] 
    data['SG_kWh_'+str(lag)] = df['SG_kWh'] 
    data['DEG_C_kWh_'+str(lag)] = df['DEG_C_kWh'] 

    return data
data_mean = add_lags(data_mean, 3)
data_mean = add_lags(data_mean, 7)
data_mean

In [ ]:
corr  = data_mean.corr()
fig = plt.figure(figsize=(30,15))
sns.heatmap(corr, annot=True)
plt.show()

From these we remove those that have a correaltion of less than 0.1 and more than 0.80
1. month vs week of year: remove weekofyear
2. month vs day of year: remove day of year
3. empty_day_mean vs empty_day_sum: remove empty_day_sum
4. full_day_sum vs full_day_mean: remove full_day_mean
5. dayofweek vs weekday:  remove dayofweek
6. experiment vs year: remove EXPERIMENT
7. remove also: holiday, school holiday
8. remove also DEC_HUM: it does not contribute too much
9. we added lags but there was not improvement in the pearson correlation and such tehy were discarded
10. we added also sav filter, for smooting and derivative. derivative and smotther of 3, 7, 29 and 89 days did not gave any results.
11. we tried tha sav filer and those features with 29 worked verywell

In [ ]:
#clean correalation plot
ALL_NUMERICAL_FEATURES = [
    'EMPTY_DAY_INT',
    'FULL_DAY_INT',
    'INCOME_SUM',
    'FULL_DAY_SUM',
    'INCOME_MEAN',
    'EMPTY_DAY_MEAN',
    'ISO_kWh',
    'DEG_C_kWh',
    'SG_kWh',
    'ISO_kWh_smooth_29',
    'SG_kWh_smooth_29',
    'DEG_C_kWh_smooth_29'
]

ALL_CATEGORICAL_FEATURES = [
    'teaching_time',
    'weekday',
    'month',
    'year',
]
corr  = data_mean[['CONSUMPTION_kWh']+ALL_NUMERICAL_FEATURES+ALL_CATEGORICAL_FEATURES].corr()
fig = plt.figure(figsize=(20,15))
sns.heatmap(corr, annot=True)
plt.show()

In [ ]:
sns.pairplot(data_mean[['CONSUMPTION_kWh']+ALL_NUMERICAL_FEATURES+ALL_CATEGORICAL_FEATURES])